# SQL Murder Mystery

In [1]:
import sqlite3
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [2]:
db_path = "../data/sql-murder-mystery.db"

conn = sqlite3.connect(db_path)

In [3]:
df_person = pd.read_sql_query("SELECT * FROM person", conn)
df_dl = pd.read_sql_query("SELECT * FROM drivers_license", conn)
df_interview = pd.read_sql_query("SELECT * FROM interview", conn)
df_report = pd.read_sql_query("SELECT * FROM crime_scene_report", conn)
df_gym = pd.read_sql_query("SELECT * FROM get_fit_now_member", conn)
df_checkin = pd.read_sql_query("SELECT * FROM get_fit_now_check_in", conn)
df_fb = pd.read_sql_query("SELECT * FROM facebook_event_checkin", conn)
conn.close()

## Checking data frames

In [4]:
df_person.head()

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684


In [5]:
df_dl.head()

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS


In [6]:
df_interview.head()

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for YOU, and no room at all\n"
4,33856,\n


In [7]:
df_report.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


In [8]:
df_gym.head()

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver


In [9]:
df_checkin.head()

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019


In [10]:
df_fb.head()

,person_id,event_id,event_name,date
0,28508,5880,Nudists are people who wear one-button suits.\n,20170913
1,63713,3865,but that's because it's the best book on anything for the layman.\n,20171009
2,63713,3999,"If Murphy's Law can go wrong, it will.\n",20170502
3,63713,6436,Old programmers never die. They just branch to a new address.\n,20170926
4,82998,4470,Help a swallow land at Capistrano.\n,20171022


## Crime Scene Report

In [11]:
# SELECT description FROM report
# WHERE date = 20180118

# AND: &
# OR: |

df_report[(df_report["date"] == 20180115)
          & (df_report["city"] == 'SQL City')
          & (df_report["type"] == 'murder')]

,date,type,description,city
1227,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


## Finding the witnesses

In [12]:
# The first witness lives at the last house on "Northwestern Dr".

max_address_number = df_person[df_person["address_street_name"] == 'Northwestern Dr']["address_number"].max()

df_person[(df_person["address_street_name"] == 'Northwestern Dr')
         & (df_person["address_number"] == max_address_number)]

,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [13]:
# The second witness, named Annabel, lives somewhere on "Franklin Ave".

df_person[(df_person["address_street_name"] == 'Franklin Ave')
         & (df_person["name"].str.contains('Annabel'))]

,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [14]:
# Finding both witnesses in one command

df_witness = df_person[((df_person["address_street_name"] == 'Northwestern Dr')
         & (df_person["address_number"] == max_address_number))
         |
         ((df_person["address_street_name"] == 'Franklin Ave')
         & (df_person["name"].str.contains('Annabel')))]

df_witness

,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


## Merging with Interview

In [15]:
# SELECT * FROM witness
# JOIN interview ON witness.id = interview.person_id

df = df_witness.merge(df_interview, how="inner", left_on="id", right_on="person_id")

# alternatively:
# df = df_interview.merge(df_witness, how="inner", left_on="person_id", right_on="id")

df

,id,name,license_id,address_number,address_street_name,ssn,person_id,transcript
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


## Finding Murderer

In [16]:
# He had a "Get Fit Now Gym" bag.
# The membership number on the bag started with "48Z".
# Only gold members have those bags.
# The man got into a car with a plate that included "H42W".
# I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


In [17]:
df_gym = df_gym.merge(df_checkin, how="inner", left_on="id", right_on="membership_id")
df_gym.head()

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,65076,Everette Koepke,20170926,gold,NL318,20180212,329,365
1,NL318,65076,Everette Koepke,20170926,gold,NL318,20170811,469,920
2,NL318,65076,Everette Koepke,20170926,gold,NL318,20180429,506,554
3,NL318,65076,Everette Koepke,20170926,gold,NL318,20180128,124,759
4,NL318,65076,Everette Koepke,20170926,gold,NL318,20171027,418,1019


In [18]:
df_gym_filtered = df_gym[df_gym["id"].str.startswith("48Z")
      & (df_gym["membership_status"] == 'gold')
      & (df_gym["check_in_date"] == 20180109)]

df = df_gym_filtered.merge(df_person, how="inner", left_on="person_id", right_on="id")
df.head()

,id_x,person_id,name_x,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time,id_y,name_y,license_id,address_number,address_street_name,ssn
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [19]:
df = df.merge(df_dl, how="inner", left_on="license_id", right_on="id")
df.head()

,id_x,person_id,name_x,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time,id_y,...,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700,67318,...,871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


In [20]:
df[df["plate_number"].str.contains("H42W")]

,id_x,person_id,name_x,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time,id_y,...,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700,67318,...,871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


## Finding Person Behind Murder

In [21]:
df_interview[df_interview["person_id"] == 67318]

,person_id,transcript
4990,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [22]:
# I was hired by a woman with a lot of money.
# I don't know her name but I know she's around 5'5" (65") or 5'7" (67").
# She has red hair and she drives a Tesla Model S.
# I know that she attended the SQL Symphony Concert 3 times in December 2017.

df = df_person.merge(df_dl, how="inner", left_on="license_id", right_on="id")
df.head()


,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076,993845,46,59,black,green,male,557472,Chrysler,Town & Country
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044,861794,54,74,black,white,female,3P6DMS,BMW,M Roadster
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512,385336,24,79,blue,green,female,GM6Y5J,Mercedes-Benz,CLS-Class
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061,431897,22,75,amber,red,female,461S1M,Ford,Ranger
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684,550890,45,60,blue,black,female,8030O8,Toyota,Tacoma Xtra


In [23]:
df = df.merge(df_fb, left_on="id_x", right_on="person_id")
df.head()

,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,person_id,event_id,event_name,date
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076,993845,46,59,black,green,male,557472,Chrysler,Town & Country,10000,241,Steinbach's Guideline for Systems Programming\n,20170306
1,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076,993845,46,59,black,green,male,557472,Chrysler,Town & Country,10000,1137,The Universe is laughing behind your back\n,20171130
2,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044,861794,54,74,black,white,female,3P6DMS,BMW,M Roadster,10007,6447,Green light in A.M. for new projects. Red light in P.M. for traffic\n,20170925
3,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044,861794,54,74,black,white,female,3P6DMS,BMW,M Roadster,10007,4273,Modern man is the missing link between apes and human beings.\n,20171017
4,10010,Muoi Cary,385336,741,Northwestern Dr,828638512,385336,24,79,blue,green,female,GM6Y5J,Mercedes-Benz,CLS-Class,10010,3581,upon to act in accordance with the dictates of reason.\n,20180319


In [24]:
df_copy = df[(df["gender"] == 'female')
  & (df["hair_color"] == 'red')
  & (df["height"].between(65, 67))
  & (df["car_make"] == 'Tesla')
  & (df["car_model"] == 'Model S')]
df_copy

,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,person_id,event_id,event_name,date
19955,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,99716,1143,SQL Symphony Concert,20171206
19956,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,99716,1143,SQL Symphony Concert,20171212
19957,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,99716,1143,SQL Symphony Concert,20171229


In [25]:
df_copy.groupby(["event_name"])[["event_name"]].count()

,event_name
event_name,
SQL Symphony Concert,3


In [26]:
type(df["event_name"].unique())

numpy.ndarray

In [27]:
type(df["event_name"].unique().tolist())

list